In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1227721,2021-04-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1227722,2021-04-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1227723,2021-05-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1227724,2021-05-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
32791,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32793,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32795,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32797,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32799,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
755504,2021-04-29,Arkansas,Arkansas,5001,2093,36.00,5000,Arkansas,AR,Arkansas,State,3017804
755506,2021-04-30,Arkansas,Arkansas,5001,2094,36.00,5000,Arkansas,AR,Arkansas,State,3017804
755508,2021-05-01,Arkansas,Arkansas,5001,2095,36.00,5000,Arkansas,AR,Arkansas,State,3017804
755510,2021-05-02,Arkansas,Arkansas,5001,2095,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1227721,2021-04-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1227722,2021-04-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1227723,2021-05-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1227724,2021-05-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-03') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
468,2021-05-03,Snohomish,Washington,36303,568.00,53061,WA,County,822083,4415.98,69.09
934,2021-05-03,Cook,Illinois,537224,10542.00,17031,IL,County,5150233,10431.06,204.69
1399,2021-05-03,Orange,California,270345,4969.00,6059,CA,County,3175692,8512.95,156.47
1863,2021-05-03,Maricopa,Arizona,538633,9882.00,4013,AZ,County,4485414,12008.55,220.31
2327,2021-05-03,Los Angeles,California,1234648,24209.00,6037,CA,County,10039107,12298.38,241.15
...,...,...,...,...,...,...,...,...,...,...,...
1227037,2021-05-03,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1227240,2021-05-03,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1227412,2021-05-03,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1227580,2021-05-03,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
468,2021-05-03,Snohomish,Washington,36303,568.00,53061,WA,County,822083,4415.98,69.09,69.09,4415.98
934,2021-05-03,Cook,Illinois,537224,10542.00,17031,IL,County,5150233,10431.06,204.69,204.69,10431.06
1399,2021-05-03,Orange,California,270345,4969.00,6059,CA,County,3175692,8512.95,156.47,156.47,8512.95
1863,2021-05-03,Maricopa,Arizona,538633,9882.00,4013,AZ,County,4485414,12008.55,220.31,220.31,12008.55
2327,2021-05-03,Los Angeles,California,1234648,24209.00,6037,CA,County,10039107,12298.38,241.15,241.15,12298.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227037,2021-05-03,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1227240,2021-05-03,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1227412,2021-05-03,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1227580,2021-05-03,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
789995,2021-05-03,Hale,Alabama,2217,76.00,1065,AL,County,14651,15132.07,518.74,518.74,15132.07,1
859915,2021-05-03,Clarke,Alabama,3503,60.00,1025,AL,County,23622,14829.40,254.00,254.00,14829.40,2
618081,2021-05-03,Lowndes,Alabama,1384,53.00,1085,AL,County,9726,14229.90,544.93,544.93,14229.90,3
459846,2021-05-03,Franklin,Alabama,4238,82.00,1059,AL,County,31362,13513.17,261.46,261.46,13513.17,4
548022,2021-05-03,Etowah,Alabama,13796,352.00,1055,AL,County,102268,13490.05,344.19,344.19,13490.05,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163751,2021-05-03,Platte,Wyoming,623,11.00,56031,WY,County,8393,7422.85,131.06,131.06,7422.85,19
753907,2021-05-03,Converse,Wyoming,996,17.00,56009,WY,County,13822,7205.90,122.99,122.99,7205.90,20
937332,2021-05-03,Lincoln,Wyoming,1398,12.00,56023,WY,County,19830,7049.92,60.51,60.51,7049.92,21
1043258,2021-05-03,Niobrara,Wyoming,157,2.00,56027,WY,County,2356,6663.84,84.89,84.89,6663.84,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
618081,2021-05-03,Lowndes,Alabama,1384,53.00,1085,AL,County,9726,14229.90,544.93,544.93,14229.90,3,1
789995,2021-05-03,Hale,Alabama,2217,76.00,1065,AL,County,14651,15132.07,518.74,518.74,15132.07,1,2
262837,2021-05-03,Walker,Alabama,7205,279.00,1127,AL,County,63521,11342.71,439.22,439.22,11342.71,28,3
546402,2021-05-03,Clay,Alabama,1533,56.00,1027,AL,County,13235,11582.92,423.12,423.12,11582.92,21,4
666558,2021-05-03,Greene,Alabama,922,34.00,1063,AL,County,8111,11367.28,419.18,419.18,11367.28,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664945,2021-05-03,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
937332,2021-05-03,Lincoln,Wyoming,1398,12.00,56023,WY,County,19830,7049.92,60.51,60.51,7049.92,21,20
913968,2021-05-03,Uinta,Wyoming,2195,12.00,56041,WY,County,20226,10852.37,59.33,59.33,10852.37,5,21
261604,2021-05-03,Teton,Wyoming,3739,9.00,56039,WY,County,23464,15935.05,38.36,38.36,15935.05,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.09,4415.98,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.09,4415.98,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.09,4415.98,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.09,4415.98,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.09,4415.98,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224844,2021-04-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1224845,2021-04-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1224846,2021-05-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1224847,2021-05-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
786719,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15132.07,2,1,1.00
786720,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15132.07,2,1,0.00
786721,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15132.07,2,1,0.00
786722,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15132.07,2,1,0.00
786723,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,518.74,15132.07,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953016,2021-04-29,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
953017,2021-04-30,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
953018,2021-05-01,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
953019,2021-05-02,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
615605,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,1.00,0.00
615606,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,0.00,0.00
615607,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,0.00,0.00
615608,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,0.00,0.00
615609,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613580,2021-04-29,Albany,Wyoming,4225,11.00,56001,WY,County,38880,10866.77,28.29,28.29,10972.22,23,4,6.00,0.00
613581,2021-04-30,Albany,Wyoming,4242,11.00,56001,WY,County,38880,10910.49,28.29,28.29,10972.22,23,4,17.00,0.00
613582,2021-05-01,Albany,Wyoming,4242,11.00,56001,WY,County,38880,10910.49,28.29,28.29,10972.22,23,4,0.00,0.00
613583,2021-05-02,Albany,Wyoming,4242,11.00,56001,WY,County,38880,10910.49,28.29,28.29,10972.22,23,4,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-03') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
313769,2021-05-03,Imperial,California,28275,721.00,6025,CA,County,181215,15603.01,397.87,397.87,15603.01,1,2,13.00,1.00
2327,2021-05-03,Los Angeles,California,1234648,24209.00,6037,CA,County,10039107,12298.38,241.15,241.15,12298.38,2,5,213.00,263.00
156225,2021-05-03,San Bernardino,California,296146,4655.00,6071,CA,County,2180085,13584.15,213.52,213.52,13584.15,3,4,57.00,3.00
549594,2021-05-03,Inyo,California,1422,38.00,6027,CA,County,18039,7882.92,210.65,210.65,7882.92,4,24,1.00,0.00
69853,2021-05-03,Stanislaus,California,61546,1046.00,6099,CA,County,550660,11176.77,189.95,189.95,11176.77,5,9,116.00,2.00
32367,2021-05-03,Riverside,California,298572,4574.00,6065,CA,County,2470546,12085.26,185.14,185.14,12085.26,6,6,122.00,0.00
86610,2021-05-03,Tulare,California,49477,837.00,6107,CA,County,466195,10612.94,179.54,179.54,10612.94,7,10,0.00,2.00
55575,2021-05-03,San Joaquin,California,72672,1365.00,6077,CA,County,762148,9535.16,179.10,179.10,9535.16,8,19,194.00,19.00
31521,2021-05-03,Fresno,California,101372,1678.00,6019,CA,County,999101,10146.32,167.95,167.95,10146.32,9,14,46.00,0.00
498049,2021-05-03,Merced,California,31657,454.00,6047,CA,County,277680,11400.53,163.50,163.50,11400.53,10,8,63.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1163302,2021-05-03,Lassen,California,5692,24.00,6035,CA,County,30573,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00
313769,2021-05-03,Imperial,California,28275,721.00,6025,CA,County,181215,15603.01,397.87,397.87,15603.01,1,2,13.00,1.00
666500,2021-05-03,Kings,California,22910,246.00,6031,CA,County,152940,14979.73,160.85,160.85,14979.73,11,3,5.00,0.00
156225,2021-05-03,San Bernardino,California,296146,4655.00,6071,CA,County,2180085,13584.15,213.52,213.52,13584.15,3,4,57.00,3.00
2327,2021-05-03,Los Angeles,California,1234648,24209.00,6037,CA,County,10039107,12298.38,241.15,241.15,12298.38,2,5,213.00,263.00
32367,2021-05-03,Riverside,California,298572,4574.00,6065,CA,County,2470546,12085.26,185.14,185.14,12085.26,6,6,122.00,0.00
196425,2021-05-03,Kern,California,108652,1354.00,6029,CA,County,900202,12069.74,150.41,150.41,12069.74,14,7,24.00,0.00
498049,2021-05-03,Merced,California,31657,454.00,6047,CA,County,277680,11400.53,163.50,163.50,11400.53,10,8,63.00,0.00
69853,2021-05-03,Stanislaus,California,61546,1046.00,6099,CA,County,550660,11176.77,189.95,189.95,11176.77,5,9,116.00,2.00
86610,2021-05-03,Tulare,California,49477,837.00,6107,CA,County,466195,10612.94,179.54,179.54,10612.94,7,10,0.00,2.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
313769,2021-05-03,Imperial,California,28275,721.00,6025,CA,County,181215,15603.01,397.87,397.87,15603.01,1,2,13.00,1.00
2327,2021-05-03,Los Angeles,California,1234648,24209.00,6037,CA,County,10039107,12298.38,241.15,241.15,12298.38,2,5,213.00,263.00
156225,2021-05-03,San Bernardino,California,296146,4655.00,6071,CA,County,2180085,13584.15,213.52,213.52,13584.15,3,4,57.00,3.00
549594,2021-05-03,Inyo,California,1422,38.00,6027,CA,County,18039,7882.92,210.65,210.65,7882.92,4,24,1.00,0.00
69853,2021-05-03,Stanislaus,California,61546,1046.00,6099,CA,County,550660,11176.77,189.95,189.95,11176.77,5,9,116.00,2.00
32367,2021-05-03,Riverside,California,298572,4574.00,6065,CA,County,2470546,12085.26,185.14,185.14,12085.26,6,6,122.00,0.00
86610,2021-05-03,Tulare,California,49477,837.00,6107,CA,County,466195,10612.94,179.54,179.54,10612.94,7,10,0.00,2.00
55575,2021-05-03,San Joaquin,California,72672,1365.00,6077,CA,County,762148,9535.16,179.10,179.10,9535.16,8,19,194.00,19.00
31521,2021-05-03,Fresno,California,101372,1678.00,6019,CA,County,999101,10146.32,167.95,167.95,10146.32,9,14,46.00,0.00
498049,2021-05-03,Merced,California,31657,454.00,6047,CA,County,277680,11400.53,163.50,163.50,11400.53,10,8,63.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.87,15603.01,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.87,15603.01,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.87,15603.01,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.87,15603.01,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.87,15603.01,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4112,04/29/21,Lassen,5692,24.00,18617.73,78.50,78.50,18617.73,34,1,7.00,0.00
4113,04/30/21,Lassen,5692,24.00,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00
4114,05/01/21,Lassen,5692,24.00,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00
4115,05/02/21,Lassen,5692,24.00,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.87,15603.01,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.87,15603.01,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.87,15603.01,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.87,15603.01,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.87,15603.01,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4198,04/29/21,Merced,31557,454.00,11364.52,163.50,163.50,11400.53,10,8,28.00,0.00
4199,04/30/21,Merced,31594,454.00,11377.85,163.50,163.50,11400.53,10,8,37.00,0.00
4200,05/01/21,Merced,31594,454.00,11377.85,163.50,163.50,11400.53,10,8,0.00,0.00
4201,05/02/21,Merced,31594,454.00,11377.85,163.50,163.50,11400.53,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)